In [1]:
from handyhelpers import util_aliyun, util_linux
import time

# bootstrap
bootstrap = util_aliyun.Bootstrap("worksapce.config.json")
bootstrap.run()

biz_tags = [{"Key": "project", "Value": "rocksdb"}]
aliyun_ecs_manager = util_aliyun.AliyunECSManager()

In [2]:
aliyun_ecs_manager.clean(biz_tags)

settings = aliyun_ecs_manager.create_default_settings()
settings.set_biz_tags(biz_tags)
settings.set_instance_type("ecs.g6.large")

vm_property = aliyun_ecs_manager.create_instance(settings)

In [4]:
vm_property = aliyun_ecs_manager.get_instances(biz_tags)[0]

cmd_content = """
sudo yum install -y gflags* snappy* libzstd* unzip g++
sudo wget -O /usr/local/bin/db_bench https://deployment-kits.oss-ap-southeast-1-internal.aliyuncs.com/rocksdb/db_bench
sudo chmod +x /usr/local/bin/db_bench
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=cmd_content, timeout=6000)

't-sgp4ftn0h8t6j9c'

In [ ]:
# FIO Test

vm_property = aliyun_ecs_manager.get_instances(biz_tags)[0]

cmd_content = """
fio -numjobs=1 -iodepth=32 -direct=1 -ioengine=libaio -sync=1 \
-rw=randwrite  \
-bs=4K -size=1G \
-time_based -runtime=60 -name=Fio -directory=/data1 2> /dev/null
"""
invocation_id = aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=cmd_content, timeout=6000)

In [ ]:
print(aliyun_ecs_manager.query_invocation_result(invocation_id))

In [14]:
# DB Bench Test
vm_property = aliyun_ecs_manager.get_instances(biz_tags)[0]

cmd_content = """
#!/bin/bash
sudo mkdir /data1

# 定义数据库目录，确保该目录在要测试的磁盘上
DB_DIR="/data1"

QPS=2000
DURATION=600
NUM_THREADS=16          # 并发线程数

# 定义测试参数
NUM_KEYS=$(($QPS * $DURATION / $NUM_THREADS)) # 根据QPS、并发度和测试时长计算测试量
VALUE_SIZE=1024         # 值大小设置为1KB
BENCHMARKS="fillrandom,readrandom" # 定义要进行的基准测试类型

# 清空缓存（需要root权限），以确保测试直接对磁盘进行
# 注意：这可能对生产系统造成影响，请谨慎操作
sudo sh -c "echo 3 > /proc/sys/vm/drop_caches"

# 运行db_bench测试
db_bench \
    --db=$DB_DIR \
    --num=$NUM_KEYS \
    --benchmarks=$BENCHMARKS \
    --value_size=$VALUE_SIZE \
    --threads=$NUM_THREADS \
    --duration=$DURATION \
    --cache_size=0 \
    --cache_numshardbits=-1 \
    --row_cache_size=0 \
    --use_direct_reads \
    --use_direct_io_for_flush_and_compaction \
    --histogram=1 \
    --statistics=1 2> /dev/null
"""
invocation_id = aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=cmd_content, timeout=6000)

In [16]:
print(aliyun_ecs_manager.query_invocation_result(invocation_id))

mkdir: cannot create directory ‘/data1’: File exists



In [ ]:
# aliyun_ecs_manager.clean(biz_tags)